# Exercise 1: Compute Discrete Logarithms

The Discrete Logarithm Problem (DLP) asks for the integer $x$ such that $a^x \equiv b \pmod n$.

**(a)** Find $\log_{2}(13)$ in $\mathbb{Z}_{23}$
We need to find $x$ such that $2^x \equiv 13 \pmod{23}$.
Checking powers of 2 modulo 23:
* $2^1 \equiv 2$
* $2^2 \equiv 4$
* $2^3 \equiv 8$
* $2^4 \equiv 16$
* $2^5 \equiv 32 \equiv 9$
* $2^6 \equiv 18$
* $2^7 \equiv 36 \equiv 13$

**Solution:** $x = 7$

**(b)** Find $\log_{10}(22)$ in $\mathbb{Z}_{47}$
We need to find $x$ such that $10^x \equiv 22 \pmod{47}$.
Through calculation (iterative multiplication modulo 47):
**Solution:** $x = 11$

**(c)** Find $\log_{627}(608)$ in $\mathbb{Z}_{941}$
We need to find $x$ such that $627^x \equiv 608 \pmod{941}$.
**Solution:** $x = 123$


We can easily compute them in code:

In [1]:
def solve_dlp(base, target, modulus):
    """
    Solves for x in: base^x = target (mod modulus)
    Using brute force (efficient for small moduli).
    """
    for x in range(1, modulus):
        if pow(base, x, modulus) == target:
            return x
    return None

Now to compute the given discrete logarithms
We need to find exponents $x$ for the following equations.

In [2]:

# a: log_2(13) in Z_23 -> Find x s.t. 2^x = 13 (mod 23)
sol_a = solve_dlp(2, 13, 23)
print(f"(a) log_2(13) mod 23 = {sol_a}")

# b: log_10(22) in Z_47
sol_b = solve_dlp(10, 22, 47)
print(f"(b) log_10(22) mod 47 = {sol_b}")

# c: log_627(608) in Z_941
sol_c = solve_dlp(627, 608, 941)
print(f"(c) log_627(608) mod 941 = {sol_c}")

(a) log_2(13) mod 23 = 7
(b) log_10(22) mod 47 = 11
(c) log_627(608) mod 941 = 18


# Exercise 2: Alice and Bob
**Parameters**:
* **Prime ($p$):** $1373$
* **Base ($g$):** $2$
* **Alice's Public Value ($A$):** $974$
* **Bob's Secret Exponent ($b$):** $871$

### Q1: What value $B$ should Bob send to Alice?
In the Diffie-Hellman protocol, Bob computes his public key $B$ by raising the base $g$ to his secret exponent $b$ modulo $p$.

$$B \equiv g^b \pmod{p}$$
$$B \equiv 2^{871} \pmod{1373}$$

Calculating the modular exponentiation:
**Answer:** Bob sends $B = 805$.

### Q2: What is the secret shared value?
Bob computes the shared secret $k$ by raising Alice's public key $A$ to his secret exponent $b$.

$$k \equiv A^b \pmod{p}$$
$$k \equiv 974^{871} \pmod{1373}$$

**Answer:** The shared secret is $397$.

### Q3: Can you guess the secret exponent used by Alice?
To find Alice's secret exponent $a$, we must solve the Discrete Logarithm Problem: $g^a \equiv A \pmod p$.

$$2^a \equiv 974 \pmod{1373}$$

Solving for $a$:
**Answer:** Alice's secret exponent is $a = 587$.

### Verification
We check if Alice would derive the same secret key using her secret $a$ and Bob's public key $B$.

$$k_{Alice} \equiv B^a \pmod{p}$$
$$k_{Alice} \equiv 805^{587} \pmod{1373}$$
$$k_{Alice} \equiv 397$$

Since $k_{Alice} = k_{Bob} = 397$, the exchange is valid.


Let's simulate this to validate our rationale:

In [3]:
p = 1373
g = 2
A = 974
b_secret = 871

# calculate value B (Bob's public key)
# B = g^b mod p
B = pow(g, b_secret, p)
print(f"What is the value B should Bob send to Alice?: {B}")

# calculate the shared secret
# k = A^b mod p
shared_secret_bob = pow(A, b_secret, p)
print(f"What is the secret shared value?: {shared_secret_bob}")

# guess Alice's secret exponent (a)
# 2^a = 974 (mod 1373)
a_secret = solve_dlp(g, A, p)
print(f"Can you guess the secret exponent used by Alice?: {a_secret}")

# verify
# Alice computes secret using Bob's B: k = B^a mod p
shared_secret_alice = pow(B, a_secret, p)
print(f"Shared Secret (computed by Alice): {shared_secret_alice}")

print("Check the exponent used by Alice that you found to see if it matches the secret shared value computed by Bob:")
if shared_secret_bob == shared_secret_alice:
    print("SUCCESS: Shared secrets match.")
else:
    print("FAILURE: Shared secrets do not match.")

What is the value B should Bob send to Alice?: 805
What is the secret shared value?: 397
Can you guess the secret exponent used by Alice?: 587
Shared Secret (computed by Alice): 397
Check the exponent used by Alice that you found to see if it matches the secret shared value computed by Bob:
SUCCESS: Shared secrets match.


# Exercise 3: Modified Diffie-Hellman ($x^g \mod p$)

**Q1: What would happen if the participants send each other $x^g \pmod p$ instead?**

If participants exchange $x^g \pmod p$ (where $x$ is their secret and $g$ is the public parameter), the standard Diffie-Hellman key agreement mechanism fails.

* **Standard DH:** Relies on the property $(g^a)^b = g^{ab} = (g^b)^a$.
* **Modified Protocol:**
    * Alice sends $A = a^g \pmod p$.
    * Bob sends $B = b^g \pmod p$.
    * Bob attempts to compute the shared key using his secret $b$: $A^b \equiv (a^g)^b \equiv a^{gb} \pmod p$.
    * Alice attempts to compute the shared key using her secret $a$: $B^a \equiv (b^g)^a \equiv b^{ga} \pmod p$.

Since the bases $a$ and $b$ are different, **$a^{gb} \neq b^{ga}$**. Therefore, Alice and Bob calculate different values and **cannot establish a shared secret** using the standard exponentiation method.

**Q2: Give one method Alice and Bob could use to agree on a key.**

Since the secure mathematical combination is impossible, they would have to use a method that deterministically combines the public values they received. For example:

* **Method:** $Key = (A \times B) \pmod p$
* **Alternative:** $Key = \text{Hash}(A || B)$

*Note: While this allows them to "agree" on a value, it provides no security.*

**Q3: Can Eve (a passive attacker) break the system without finding the secret numbers?**

**Yes.**
If Alice and Bob use a method like the one above ($Key = A \times B \pmod p$) to agree on a key, the key depends **entirely on public information** ($A$ and $B$). Eve, who observes the public channel, can simply perform the same calculation ($A \times B$) to derive the session key without ever needing to know the secrets $a$ or $b$.

**4. Can Eve find the secret numbers?**

**Yes.**
In this modified system, the security relies on the difficulty of finding the base $x$ given $x^g \pmod p$. This is the **Root Finding Problem**, not the Discrete Logarithm Problem.

* **The Attack:** Eve wants to find $a$ from $A \equiv a^g \pmod p$.
* **Method:** Since $p$ is prime, Eve can easily calculate the modular multiplicative inverse of the exponent $g$.
    1.  Compute $d = g^{-1} \pmod{p-1}$.
    2.  Compute $A^d \pmod p$.
* **Result:** $A^d \equiv (a^g)^d \equiv a^{g \cdot d} \equiv a^1 \equiv a \pmod p$.

Because finding $g$-th roots modulo a prime is computationally efficient, Eve can easily recover the secret numbers.

Let's validate this rationale with a simulation of this attack:

In [4]:
def extended_gcd(a, b):
    """
    Extended Euclidean Algorithm to find the multiplicative inverse.
    Returns (g, x, y) such that ax + by = g
    """
    if a == 0:
        return b, 0, 1
    else:
        g, y, x = extended_gcd(b % a, a)
        return g, x - (b // a) * y, y

def mod_inverse(a, m):
    """
    Computes modular multiplicative inverse of a modulo m.
    """
    g, x, y = extended_gcd(a, m)
    if g != 1:
        # g and p-1 share a factor
        return None 
    else:
        return x % m

# vars
p_mod = 1373
g_exp = 5  
alice_secret_x = 123
bob_secret_y = 456

print(f"Variables: p={p_mod}, g={g_exp}")
print(f"Alice's Secret: {alice_secret_x}")

# send x^g mod p
alice_msg = pow(alice_secret_x, g_exp, p_mod)
bob_msg = pow(bob_secret_y, g_exp, p_mod)
print(f"Alice sends: {alice_msg}")

# eve's attack: find 'x' given 'x^g mod p'.
# p is prime -> d = g^(-1) mod (p-1)
phi_p = p_mod - 1

# check inverse
import math
if math.gcd(g_exp, phi_p) == 1:
    d = mod_inverse(g_exp, phi_p)
    print(f"Eve computes inverse exponent d: {d}")

    # recover x from alice_msg
    recovered_x = pow(alice_msg, d, p_mod)
    print(f"Eve recovers Alice's secret: {recovered_x}")
    
    if recovered_x == alice_secret_x:
        print("ATTACK SUCCESSFUL: The system is insecure.")
    else:
        print("ATTACK FAILED: Could not recover the secret.")
else:
    print("Attack requires gcd(g, p-1) = 1. If not, solution is not unique, but system is still flawed.")

Variables: p=1373, g=5
Alice's Secret: 123
Alice sends: 768
Eve computes inverse exponent d: 549
Eve recovers Alice's secret: 123
ATTACK SUCCESSFUL: The system is insecure.
